In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
import random
from typing import Dict, List, Tuple, Set
from collections import defaultdict

In [ ]:
class TemporalLinkDataset:
    """Dataset with improved negative sampling using node corruption from actual edges."""

    def __init__(self, edges_df: pd.DataFrame, edge_features: torch.Tensor,
                 sequence_length: int = 10):
        """
        Args:
            edges_df: DataFrame with columns [src, dst, time, ext_roll] (ALL POSITIVE SAMPLES)
            edge_features: Tensor of shape [num_edges, 2] with src and dst features
            sequence_length: Length of temporal sequence to use for prediction
        """
        self.edges_df = edges_df.copy()
        self.edge_features = edge_features
        self.sequence_length = sequence_length

        # Verify alignment
        assert len(self.edges_df) == len(self.edge_features), \
            f"Mismatch: {len(self.edges_df)} edges vs {len(self.edge_features)} features"

        # Sort by time for temporal consistency
        self.edges_df = self.edges_df.sort_values('time').reset_index(drop=True)
        # Reorder features to match the sorted edges
        original_indices = self.edges_df.index.tolist()  # This gives us the mapping
        # Actually, after reset_index(drop=True), we need to track original order
        # Let's be more careful:

        # Store original index before sorting
        self.edges_df['original_idx'] = self.edges_df.index
        self.edges_df = self.edges_df.sort_values('time').reset_index(drop=True)

        # Reorder features to match sorted edges
        sorted_feature_indices = self.edges_df['original_idx'].tolist()
        self.edge_features = self.edge_features[sorted_feature_indices]

        # Now drop the helper column
        self.edges_df = self.edges_df.drop('original_idx', axis=1)

        # Get nodes that appear in each time split for realistic negative sampling
        self.split_nodes = self._get_split_nodes()

        # Create positive edge sets for each timestamp within each split
        self.positive_edges_by_time = self._create_positive_edges_by_time()

        # Generate negative samples using node corruption
        self.samples = self._generate_samples_with_negatives()

        print(f"Created dataset with {len(self.samples)} samples")
        pos_count = sum(1 for s in self.samples if s['target'] == 1)
        neg_count = sum(1 for s in self.samples if s['target'] == 0)
        print(f"Positive samples: {pos_count}")
        print(f"Negative samples: {neg_count}")

    def _get_split_nodes(self) -> Dict[int, Set[int]]:
        """Get all nodes that appear in each split for realistic negative sampling."""
        split_nodes = {}
        for ext_roll in [0, 1, 2]:
            split_edges = self.edges_df[self.edges_df['ext_roll'] == ext_roll]
            nodes = set(split_edges['src'].unique()) | set(split_edges['dst'].unique())
            split_nodes[ext_roll] = nodes
        return split_nodes

    def _create_positive_edges_by_time(self) -> Dict[Tuple[int, int], Set[Tuple[int, int]]]:
        """Create positive edge sets for each (ext_roll, time) combination."""
        positive_edges = {}

        for _, row in self.edges_df.iterrows():
            key = (row['ext_roll'], row['time'])
            if key not in positive_edges:
                positive_edges[key] = set()
            positive_edges[key].add((row['src'], row['dst']))

        return positive_edges

    def _generate_samples_with_negatives(self) -> List[Dict]:
        """Generate both positive samples and negative samples using node corruption."""
        samples = []

        for idx, row in self.edges_df.iterrows():
            src, dst, time, ext_roll = row['src'], row['dst'], row['time'], row['ext_roll']

            # Get historical context (same logic as before but with corrected indexing)
            if ext_roll == 0:  # Training
                historical_mask = (self.edges_df['time'] < time) & (self.edges_df['ext_roll'] == 0)
            else:  # Val/Test - can use training + previous split data
                max_roll = 0 if ext_roll == 1 else 1
                historical_mask = (self.edges_df['time'] < time) & (self.edges_df['ext_roll'] <= max_roll)

            historical_edges = self.edges_df[historical_mask].tail(self.sequence_length)

            # Create sequence features (now properly aligned)
            if len(historical_edges) > 0:
                hist_indices = historical_edges.index.tolist()
                sequence_features = self.edge_features[hist_indices]

                # Pad if necessary
                if len(sequence_features) < self.sequence_length:
                    padding = torch.zeros(self.sequence_length - len(sequence_features), 2)
                    sequence_features = torch.cat([padding, sequence_features], dim=0)
            else:
                sequence_features = torch.zeros(self.sequence_length, 2)

            # POSITIVE SAMPLE
            current_features = self.edge_features[idx]
            samples.append({
                'sequence': sequence_features,
                'current_features': current_features,
                'target': 1,
                'ext_roll': ext_roll,
                'time': time,
                'src': src,
                'dst': dst,
                'edge_idx': idx
            })

            # NEGATIVE SAMPLE using node corruption
            negative_sample = self._create_negative_sample(src, dst, time, ext_roll, sequence_features)
            if negative_sample is not None:
                samples.append(negative_sample)

        return samples

    def _create_negative_sample(self, src: int, dst: int, time: int, ext_roll: int,
                              sequence_features: torch.Tensor) -> Dict:
        """
        Create negative sample by corrupting either src or dst node.
        Uses nodes from the same split to ensure realistic corruption.
        """
        # Get positive edges at this specific time and split
        time_key = (ext_roll, time)
        positive_edges_at_time = self.positive_edges_by_time.get(time_key, set())

        # Get available nodes from this split
        available_nodes = list(self.split_nodes[ext_roll])

        # Try to corrupt dst first (keep src, change dst)
        max_attempts = 50
        attempts = 0

        while attempts < max_attempts:
            attempts += 1

            # Randomly choose to corrupt src or dst
            if random.random() < 0.5:
                # Corrupt dst (keep src)
                neg_src = src
                neg_dst = random.choice(available_nodes)
            else:
                # Corrupt src (keep dst)
                neg_src = random.choice(available_nodes)
                neg_dst = dst

            # Check if this would be a valid negative (not in positive edges at this time)
            if ((neg_src, neg_dst) not in positive_edges_at_time and
                neg_src != neg_dst):

                # Create features for negative sample
                # We'll use features from existing edges involving these nodes
                neg_features = self._get_features_for_negative(neg_src, neg_dst, time, ext_roll)

                if neg_features is not None:
                    return {
                        'sequence': sequence_features.clone(),
                        'current_features': neg_features,
                        'target': 0,
                        'ext_roll': ext_roll,
                        'time': time,
                        'src': neg_src,
                        'dst': neg_dst,
                        'edge_idx': -1
                    }

        # If we couldn't create a valid negative sample, return None
        return None

    def _get_features_for_negative(self, neg_src: int, neg_dst: int, time: int, ext_roll: int) -> torch.Tensor:
        """
        Get features for negative sample by looking for edges involving these nodes
        around the same time period, or use interpolation/averaging.
        """
        # Strategy 1: Find edges involving neg_src or neg_dst around this time
        time_window = 5  # Look within ±5 time units

        # Look for edges involving these nodes in a time window
        mask = (
            ((self.edges_df['src'] == neg_src) | (self.edges_df['dst'] == neg_src) |
             (self.edges_df['src'] == neg_dst) | (self.edges_df['dst'] == neg_dst)) &
            (abs(self.edges_df['time'] - time) <= time_window) &
            (self.edges_df['ext_roll'] <= ext_roll)  # Only use past/current data
        )

        candidate_edges = self.edges_df[mask]

        if len(candidate_edges) > 0:
            # Use features from a random candidate edge
            candidate_idx = random.choice(candidate_edges.index.tolist())
            base_features = self.edge_features[candidate_idx].clone()

            # Add small noise to make it slightly different
            noise = torch.normal(0, 0.1, size=base_features.shape)
            return base_features + noise

        # Strategy 2: If no candidates found, use average features from the split with noise
        split_mask = self.edges_df['ext_roll'] <= ext_roll
        if split_mask.sum() > 0:
            split_features = self.edge_features[split_mask]
            mean_features = split_features.mean(dim=0)
            std_features = split_features.std(dim=0)

            # Generate features based on distribution
            return torch.normal(mean_features, std_features * 0.2)

        # Strategy 3: Fallback to zero features (should rarely happen)
        return torch.zeros(2)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        return {
            'sequence': sample['sequence'].float(),
            'current_features': sample['current_features'].float(),
            'target': torch.tensor(sample['target'], dtype=torch.float32),
            'ext_roll': sample['ext_roll']
        }

In [ ]:
class TemporalLSTMPredictor(nn.Module):
    """LSTM-based temporal link prediction model."""

    def __init__(self, input_dim: int = 2, hidden_dim: int = 64,
                 num_layers: int = 2, dropout: float = 0.2, use_gru: bool = False):
        """
        Args:
            input_dim: Dimension of edge features
            hidden_dim: Hidden dimension of LSTM/GRU
            num_layers: Number of LSTM/GRU layers
            dropout: Dropout rate
            use_gru: Whether to use GRU instead of LSTM
        """
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.use_gru = use_gru

        # Temporal sequence encoder
        if use_gru:
            self.temporal_encoder = nn.GRU(
                input_dim, hidden_dim, num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0
            )
        else:
            self.temporal_encoder = nn.LSTM(
                input_dim, hidden_dim, num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0
            )

        # Current edge feature encoder
        self.current_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )

        # Fusion and prediction layers
        self.fusion = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )

    def forward(self, sequence, current_features):
        """
        Args:
            sequence: [batch_size, seq_len, input_dim]
            current_features: [batch_size, input_dim]
        """
        batch_size = sequence.size(0)

        # Encode temporal sequence
        if self.use_gru:
            _, hidden = self.temporal_encoder(sequence)
            temporal_repr = hidden[-1]  # Use last layer's hidden state
        else:
            _, (hidden, _) = self.temporal_encoder(sequence)
            temporal_repr = hidden[-1]  # Use last layer's hidden state

        # Encode current features
        current_repr = self.current_encoder(current_features)

        # Fuse representations
        fused = torch.cat([temporal_repr, current_repr], dim=1)

        # Predict link probability
        output = self.fusion(fused)

        return output.squeeze()


In [ ]:
class TemporalLinkPredictor:
    """Enhanced trainer class for temporal link prediction with ranking metrics."""

    def __init__(self, model: nn.Module, device: str = 'cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.device = device
        self.criterion = nn.BCELoss()

    def train_epoch(self, dataloader: DataLoader, optimizer: optim.Optimizer) -> float:
        """Train for one epoch."""
        self.model.train()
        total_loss = 0

        for batch in dataloader:
            sequence = batch['sequence'].to(self.device)
            current_features = batch['current_features'].to(self.device)
            targets = batch['target'].to(self.device)

            optimizer.zero_grad()

            outputs = self.model(sequence, current_features)
            loss = self.criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        return total_loss / len(dataloader)

    def evaluate(self, dataloader: DataLoader) -> Dict[str, float]:
        """Evaluate the model with standard binary classification metrics."""
        self.model.eval()
        all_predictions = []
        all_targets = []
        total_loss = 0

        with torch.no_grad():
            for batch in dataloader:
                sequence = batch['sequence'].to(self.device)
                current_features = batch['current_features'].to(self.device)
                targets = batch['target'].to(self.device)

                outputs = self.model(sequence, current_features)
                loss = self.criterion(outputs, targets)
                total_loss += loss.item()

                all_predictions.extend(outputs.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        all_predictions = np.array(all_predictions)
        all_targets = np.array(all_targets)

        auc = roc_auc_score(all_targets, all_predictions)
        ap = average_precision_score(all_targets, all_predictions)

        return {
            'loss': total_loss / len(dataloader),
            'auc': auc,
            'ap': ap
        }

    def calculate_ranking_metrics(self, test_dataset, edges_df: pd.DataFrame) -> Dict[str, float]:
        """
        Calculate R@1, R@5, R@10, and MRR metrics for temporal link prediction.

        For each positive test edge (src, dst, time), we:
        1. Keep the source node and temporal context fixed
        2. Replace dst with all other possible nodes to create negative samples
        3. Get model predictions for all candidates
        4. Rank based on prediction scores
        5. Calculate metrics based on true destination's rank

        Args:
            test_dataset: Test dataset containing positive samples
            edges_df: Full edges dataframe to get all possible nodes

        Returns:
            Dictionary with R@1, R@5, R@10, and MRR scores
        """
        self.model.eval()

        # Get all unique nodes from the dataset
        all_nodes = set(edges_df['src'].unique()) | set(edges_df['dst'].unique())
        all_nodes = sorted(list(all_nodes))

        # Group test samples by (src, time, ext_roll) to process efficiently
        test_samples_by_context = defaultdict(list)

        # Extract positive test samples from dataset
        positive_samples = []
        for sample in test_dataset.samples:
            if sample['target'] == 1:  # Only positive samples
                positive_samples.append(sample)

                # Group by source context for efficient processing
                context_key = (sample['src'], sample['time'], sample['ext_roll'])
                test_samples_by_context[context_key].append(sample)

        print(f"Evaluating ranking metrics on {len(positive_samples)} positive test samples...")

        ranks = []

        with torch.no_grad():
            for i, (context_key, context_samples) in enumerate(test_samples_by_context.items()):
                src, time, ext_roll = context_key

                if i % 100 == 0:
                    print(f"Processing context {i+1}/{len(test_samples_by_context)}")

                # For each positive sample in this context
                for sample in context_samples:
                    true_dst = sample['dst']
                    sequence_features = sample['sequence']

                    # Create candidate pairs: (src, candidate_dst) for all possible destinations
                    candidate_scores = []
                    candidate_nodes = []

                    for candidate_dst in all_nodes:
                        if candidate_dst == src:  # Skip self-loops
                            continue

                        # Create features for this candidate pair
                        candidate_features = self._get_candidate_features(
                            src, candidate_dst, time, ext_roll,
                            test_dataset, edges_df
                        )

                        if candidate_features is not None:
                            # Get model prediction
                            sequence_batch = sequence_features.unsqueeze(0).float().to(self.device)
                            features_batch = candidate_features.unsqueeze(0).float().to(self.device)

                            score = self.model(sequence_batch, features_batch)
                            candidate_scores.append(score.item())
                            candidate_nodes.append(candidate_dst)

                    # Rank candidates by score (higher is better)
                    if len(candidate_scores) > 0:
                        scored_candidates = list(zip(candidate_nodes, candidate_scores))
                        scored_candidates.sort(key=lambda x: x[1], reverse=True)

                        # Find rank of true destination (1-indexed)
                        true_rank = None
                        for rank, (node, score) in enumerate(scored_candidates, 1):
                            if node == true_dst:
                                true_rank = rank
                                break

                        if true_rank is not None:
                            ranks.append(true_rank)
                        else:
                            # If true destination not found in candidates, assign worst rank
                            ranks.append(len(scored_candidates) + 1)

        # Calculate metrics
        ranks = np.array(ranks)

        # Recall at K
        r_at_1 = np.mean(ranks <= 1)
        r_at_5 = np.mean(ranks <= 5)
        r_at_10 = np.mean(ranks <= 10)

        # Mean Reciprocal Rank
        mrr = np.mean(1.0 / ranks)

        return {
            'R@1': r_at_1,
            'R@5': r_at_5,
            'R@10': r_at_10,
            'MRR': mrr,
            'mean_rank': np.mean(ranks),
            'median_rank': np.median(ranks),
            'num_samples': len(ranks)
        }

    def _get_candidate_features(self, src: int, dst: int, time: int, ext_roll: int,
                              test_dataset, edges_df: pd.DataFrame) -> torch.Tensor:
        """
        Get features for a candidate (src, dst) pair using the same strategy
        as the dataset's negative sampling approach.
        """
        # Use the same feature generation strategy as the dataset
        return test_dataset._get_features_for_negative(src, dst, time, ext_roll)

    def evaluate_comprehensive(self, test_dataset, edges_df: pd.DataFrame,
                             test_dataloader: DataLoader = None) -> Dict[str, float]:
        """
        Comprehensive evaluation including both standard binary classification metrics
        and ranking metrics.

        Args:
            test_dataset: Test dataset for ranking evaluation
            edges_df: Full edges dataframe
            test_dataloader: Optional dataloader for binary classification metrics

        Returns:
            Dictionary containing all metrics
        """
        print("Calculating standard binary classification metrics...")

        # Standard evaluation using dataloader if provided
        if test_dataloader is not None:
            standard_metrics = self.evaluate(test_dataloader)
        else:
            # Create temporary dataloader if not provided
            temp_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
            standard_metrics = self.evaluate(temp_loader)

        print("Calculating ranking metrics...")

        # Ranking metrics
        ranking_metrics = self.calculate_ranking_metrics(test_dataset, edges_df)

        # Combine all metrics
        all_metrics = {
            'loss': standard_metrics['loss'],
            'auc': standard_metrics['auc'],
            'ap': standard_metrics['ap'],
            **ranking_metrics
        }

        return all_metrics

    def print_evaluation_results(self, metrics: Dict[str, float]):
        """Pretty print evaluation results."""
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)

        print("\nBinary Classification Metrics:")
        if 'loss' in metrics:
            print(f"  Loss:                   {metrics['loss']:.4f}")
        print(f"  AUC:                    {metrics['auc']:.4f}")
        print(f"  Average Precision (AP): {metrics['ap']:.4f}")

        if 'R@1' in metrics:
            print("\nRanking Metrics:")
            print(f"  R@1:                   {metrics['R@1']:.4f}")
            print(f"  R@5:                   {metrics['R@5']:.4f}")
            print(f"  R@10:                  {metrics['R@10']:.4f}")
            print(f"  MRR:                   {metrics['MRR']:.4f}")

            print(f"\nRank Statistics:")
            print(f"  Mean Rank:             {metrics['mean_rank']:.2f}")
            print(f"  Median Rank:           {metrics['median_rank']:.2f}")
            print(f"  Number of Samples:     {metrics['num_samples']}")

        print("="*60)


        # """
        # Standalone function for backward compatibility.

        # Args:
        #     model: Trained temporal link prediction model
        #     test_dataset: Test dataset containing positive samples
        #     edges_df: Full edges dataframe to get all possible nodes
        #     device: Device to run inference on

        # Returns:
        #     Dictionary with R@1, R@5, R@10, and MRR scores
        # """
        # # Create a temporary trainer instance to use the class method
        # trainer = TemporalLinkPredictor(model, device)
        # return trainer.calculate_ranking_metrics(test_dataset, edges_df)


def evaluate_model_with_ranking(model: torch.nn.Module, test_dataset, edges_df: pd.DataFrame,
                               device: str = 'cuda') -> Dict[str, float]:
    """
    Standalone function for comprehensive evaluation (backward compatibility).
    """
    trainer = TemporalLinkPredictor(model, device)
    return trainer.evaluate_comprehensive(test_dataset, edges_df)


def print_evaluation_results(metrics: Dict[str, float]):
    """Standalone function for printing results (backward compatibility)."""
    # Create a dummy trainer instance to use the class method
    dummy_model = torch.nn.Linear(1, 1)  # Dummy model just for the printer
    trainer = TemporalLinkPredictor(dummy_model)
    trainer.print_evaluation_results(metrics)

In [ ]:
def create_dataloaders(edges_df: pd.DataFrame, edge_features: torch.Tensor,
                              sequence_length: int = 10, batch_size: int = 32):
    """Create dataloaders with improved negative sampling."""

    # Split data based on ext_roll
    train_df = edges_df[edges_df['ext_roll'] == 0].copy()
    val_df = edges_df[edges_df['ext_roll'] == 1].copy()
    test_df = edges_df[edges_df['ext_roll'] == 2].copy()

    # Get corresponding features
    train_features = edge_features[edges_df['ext_roll'] == 0]
    val_features = edge_features[edges_df['ext_roll'] == 1]
    test_features = edge_features[edges_df['ext_roll'] == 2]

    # Create datasets
    train_dataset = TemporalLinkDataset(train_df, train_features, sequence_length)
    val_dataset = TemporalLinkDataset(val_df, val_features, sequence_length)
    test_dataset = TemporalLinkDataset(test_df, test_features, sequence_length)

    # Create dataloaders
    from torch.utils.data import DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

def main(edges_df_path, edge_features_path):
    # Load data (same as your existing code)
    edges_df = pd.read_csv(edges_df_path)
    edge_features = torch.load(edge_features_path)

    print(f"Loaded {len(edges_df)} edges and {edge_features.shape[0]} edge features")

    # Hyperparameters
    sequence_length = 20
    hidden_dim = 128
    num_layers = 2
    dropout = 0.2
    batch_size = 128
    learning_rate = 0.001
    num_epochs = 100
    use_gru = False

    # Create dataloaders (using your existing function)
    train_loader, val_loader, test_loader = create_dataloaders(
        edges_df, edge_features, sequence_length, batch_size
    )

    # Create model (using your existing class)
    model = TemporalLSTMPredictor(
        input_dim=2,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        dropout=dropout,
        use_gru=use_gru
    )

    # Create enhanced trainer
    trainer = TemporalLinkPredictor(model)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)

    print(f"\nTraining {'GRU' if use_gru else 'LSTM'} model...")
    print(f"Model parameters: {sum(p.numel() for p in model.parameters())}")

    # Training loop
    best_val_auc = 0
    for epoch in range(num_epochs):
        # Train
        train_loss = trainer.train_epoch(train_loader, optimizer)

        # Validate
        val_metrics = trainer.evaluate(val_loader)
        scheduler.step(val_metrics['loss'])

        # Save best model
        if val_metrics['auc'] > best_val_auc:
            best_val_auc = val_metrics['auc']
            torch.save(model.state_dict(), f'best_{"gru" if use_gru else "lstm"}_model.pt')

        if epoch % 10 == 0:
            print(f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                  f"Val Loss: {val_metrics['loss']:.4f} | Val AUC: {val_metrics['auc']:.4f} | "
                  f"Val AP: {val_metrics['ap']:.4f}")

    # Load best model
    trainer.model.load_state_dict(torch.load(f'best_{"gru" if use_gru else "lstm"}_model.pt'))

    # Prepare test dataset for ranking evaluation
    test_df = edges_df[edges_df['ext_roll'] == 2].copy()
    test_features = edge_features[edges_df['ext_roll'] == 2]
    test_dataset = TemporalLinkDataset(test_df, test_features, sequence_length)

    # Comprehensive evaluation with both standard and ranking metrics
    print("\n" + "="*50)
    print("FINAL EVALUATION")
    print("="*50)

    all_metrics = trainer.evaluate_comprehensive(
        test_dataset=test_dataset,
        edges_df=edges_df,
        test_dataloader=test_loader
    )

    # Print results using the class method
    trainer.print_evaluation_results(all_metrics)

    return all_metrics, trainer

# **Positive + Negative**

In [ ]:
edges_df_path, edge_features_path = 'leadlag/edges.csv', 'leadlag/edge_features.pt'
main(edges_df_path, edge_features_path)

Loaded 9440 edges and 9440 edge features
Created dataset with 17372 samples
Positive samples: 8686
Negative samples: 8686
Created dataset with 746 samples
Positive samples: 373
Negative samples: 373
Created dataset with 762 samples
Positive samples: 381
Negative samples: 381

Training LSTM model...
Model parameters: 65409
Epoch   0 | Train Loss: 0.6971 | Val Loss: 0.6955 | Val AUC: 0.4828 | Val AP: 0.4930
Epoch  10 | Train Loss: 0.6842 | Val Loss: 0.6961 | Val AUC: 0.5007 | Val AP: 0.4922
Epoch  20 | Train Loss: 0.6587 | Val Loss: 0.6941 | Val AUC: 0.5006 | Val AP: 0.5004
Epoch  30 | Train Loss: 0.6478 | Val Loss: 0.6937 | Val AUC: 0.5214 | Val AP: 0.5208
Epoch  40 | Train Loss: 0.6424 | Val Loss: 0.6943 | Val AUC: 0.5251 | Val AP: 0.5229
Epoch  50 | Train Loss: 0.6407 | Val Loss: 0.6945 | Val AUC: 0.5261 | Val AP: 0.5231
Epoch  60 | Train Loss: 0.6397 | Val Loss: 0.6946 | Val AUC: 0.5266 | Val AP: 0.5234
Epoch  70 | Train Loss: 0.6409 | Val Loss: 0.6946 | Val AUC: 0.5267 | Val AP: 0.5

({'loss': 0.6948979496955872,
  'auc': np.float64(0.5192923719180772),
  'ap': np.float64(0.5332981736199693),
  'R@1': np.float64(0.06561679790026247),
  'R@5': np.float64(0.2204724409448819),
  'R@10': np.float64(0.38320209973753283),
  'MRR': np.float64(0.1734564224512255),
  'mean_rank': np.float64(14.729658792650918),
  'median_rank': np.float64(15.0),
  'num_samples': 381},
 <__main__.TemporalLinkPredictor at 0x7aa45d163bd0>)

# Positive only

In [ ]:
edges_df_path, edge_features_path = 'positive/edges.csv', 'positive/edge_features.pt'
main(edges_df_path, edge_features_path)

Loaded 2604 edges and 2604 edge features
Created dataset with 4660 samples
Positive samples: 2330
Negative samples: 2330
Created dataset with 338 samples
Positive samples: 169
Negative samples: 169
Created dataset with 210 samples
Positive samples: 105
Negative samples: 105

Training LSTM model...
Model parameters: 257793
Epoch   0 | Train Loss: 0.7038 | Val Loss: 0.6932 | Val AUC: 0.5040 | Val AP: 0.4990
Epoch  10 | Train Loss: 0.6908 | Val Loss: 0.6983 | Val AUC: 0.4811 | Val AP: 0.4988
Epoch  20 | Train Loss: 0.6897 | Val Loss: 0.6958 | Val AUC: 0.4815 | Val AP: 0.4917
Epoch  30 | Train Loss: 0.6827 | Val Loss: 0.6995 | Val AUC: 0.4813 | Val AP: 0.4932
Epoch  40 | Train Loss: 0.6789 | Val Loss: 0.7003 | Val AUC: 0.4808 | Val AP: 0.4921
Epoch  50 | Train Loss: 0.6798 | Val Loss: 0.7004 | Val AUC: 0.4810 | Val AP: 0.4923
Epoch  60 | Train Loss: 0.6793 | Val Loss: 0.7004 | Val AUC: 0.4809 | Val AP: 0.4923
Epoch  70 | Train Loss: 0.6789 | Val Loss: 0.7004 | Val AUC: 0.4809 | Val AP: 0.4

({'loss': 0.6919388175010681,
  'auc': np.float64(0.5239909297052154),
  'ap': np.float64(0.5236112611717691),
  'R@1': np.float64(0.009523809523809525),
  'R@5': np.float64(0.1619047619047619),
  'R@10': np.float64(0.3333333333333333),
  'MRR': np.float64(0.1214554935202401),
  'mean_rank': np.float64(15.17142857142857),
  'median_rank': np.float64(15.0),
  'num_samples': 105},
 <__main__.TemporalLinkPredictor at 0x7b4018918ad0>)